## 按格式删除文件

In [ ]:
import os
from glob import glob

# 读取路径
image_sequence_path = "/Users/WangHao/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/佳文数据_0722/短轴视频_0722"
image_sequence_list = os.listdir(image_sequence_path)
image_sequence_list = sorted(image_sequence_list)

for path_name in image_sequence_list:
    # 读取文件
    ext = "npy"
    file_list = glob(f"{os.path.join(image_sequence_path, path_name)}/*.{ext}")
    try:
        for file in file_list:
            os.remove(file)
    except:
        print(f"{path_name}中不存在{ext}类型文件")

print("运行结束")

## 按需删除文件信息
1. 02220820210305_XJ_20210305082931543.nii.gz -> 去除0-5共6帧
2. 202204020917510011VAS.nii.gz -> 去除0-28共29帧
3. IM_0044.nii.gz -> 去除73-136共64帧 

In [1]:
import os
from glob import glob
import SimpleITK as sitk

# 读取路径
data_sequence_path = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/测试集去除分叉帧数据及结果"
data_sequence_list = os.listdir(data_sequence_path)
data_sequence_list = sorted(data_sequence_list)

for folder_name in data_sequence_list:
    # 读取文件
    ext = "nii.gz"
    file_list = glob(f"{os.path.join(data_sequence_path, folder_name)}/*.{ext}")
    try:
        for file in file_list:
            if file == os.path.join(
                data_sequence_path,
                folder_name,
                "02220820210305_XJ_20210305082931543.nii.gz",
            ):
                data = sitk.GetArrayFromImage(sitk.ReadImage(file))
                data = data[6:]
                print(data.shape)
                outputs = sitk.GetImageFromArray(data)
                sitk.WriteImage(outputs, file)
            elif file == os.path.join(
                data_sequence_path, folder_name, "202204020917510011VAS.nii.gz"
            ):
                data = sitk.GetArrayFromImage(sitk.ReadImage(file))
                data = data[29:]
                print(data.shape)
                outputs = sitk.GetImageFromArray(data)
                sitk.WriteImage(outputs, file)
            elif file == os.path.join(
                data_sequence_path, folder_name, "IM_0044.nii.gz"
            ):
                data = sitk.GetArrayFromImage(sitk.ReadImage(file))
                data = data[:73]
                print(data.shape)
                outputs = sitk.GetImageFromArray(data)
                sitk.WriteImage(outputs, file)
            else:
                pass
    except:
        print(f"{file}删除出错")

print("运行结束")

(72, 203, 195)
(111, 191, 223)
(73, 136, 287)
(73, 136, 287)
(111, 191, 223)
(72, 203, 195)
(111, 128, 128)
(72, 128, 128)
(73, 128, 128)
(72, 128, 128)
(111, 128, 128)
(73, 128, 128)
运行结束


## 按标签删除文件

In [2]:
import os

# 读取路径
label_sequence_path = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/0910/label_roi第一次平滑"
label_sequence_list = os.listdir(label_sequence_path)
label_sequence_list = sorted(label_sequence_list)

image_sequence_path = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/0910/image_roi第一次平滑"
image_sequence_list = os.listdir(image_sequence_path)
image_sequence_list = sorted(image_sequence_list)

for label_name in label_sequence_list:
    try:
        if label_name not in image_sequence_list:
            # 移除文件
            label_file_name = os.path.join(label_sequence_path, label_name)
            os.remove(label_file_name)
    except:
        print(f"文件{label_file_name}不存在")

print("运行结束")

运行结束


## 修改像道数据

In [ ]:
import cv2
import numpy as np


filePath = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset_0802/label/06570820220302_CA MASHAOYONG_20220302091109144"
if not os.path.exists(filePath.replace("dataset_0706", "dataset_0706_nb")):
    os.mkdir(filePath.replace("dataset_0706", "dataset_0706_nb"))

for root, dirs, files in os.walk(filePath):
    for i in files:
        if "label" in i:
            label = cv2.imdecode(np.fromfile(f"{root}/{i}", dtype=np.uint8), -1)
            label[..., 0] = 0
            image = cv2.imread(root + "/" + i.replace("label", "image"))

            if not os.path.exists(root.replace("dataset_0706", "dataset_0706_nb")):
                os.mkdir(root.replace("dataset_0706", "dataset_0706_nb"))

            cv2.imencode(".png", label)[-1].tofile(
                root.replace("dataset_0706", "dataset_0706_nb") + "/" + i
            )
            cv2.imencode(".png", image)[-1].tofile(
                root.replace("dataset_0706", "dataset_0706_nb")
                + "/"
                + i.replace("label", "image")
            )

print("运行结束")

# 去除小连通

In [ ]:
import cv2
import numpy as np
from skimage import morphology


def remove_isolate(inputs, threshold_area=0.5):
    """
    inputs: x*y*3
    """
    mask = np.zeros((inputs.shape[0], inputs.shape[1]), dtype=np.uint8)
    mask[np.sum(inputs, axis=-1) > 0] = 1
    mask = morphology.remove_small_objects(
        mask.astype(np.bool8), np.sum(mask) * threshold_area, connectivity=8
    ).astype(np.uint8)
    outputs = np.expand_dims(mask, -1) * inputs

    return outputs


filePath = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/label"

for root, dirs, files in os.walk(filePath):
    for file_name in files:
        ext = ".png"
        if ext in file_name and "._" not in file_name:
            label = cv2.imdecode(np.fromfile(f"{root}/{file_name}", dtype=np.uint8), -1)
            label = remove_isolate(label)
            cv2.imencode(".png", label)[-1].tofile(f"{root}/{file_name}")

print("运行结束")

# 生成重叠图

In [ ]:
import os
import cv2

path = r"/Users/WangHao/Downloads/ToData"
save_path = ""

# 通过DFS实现统一逻辑处理同一层次的文件对象
for root, dirs, files in os.walk(path):
    if not dirs:
        keywords = ["label", "label_roi"]
        dir_name = root.split("/")[-1]
        keys = [key for key in keywords if key == dir_name]

        if keys == ["label"]:
            save_visual_path = root.replace("label", "visual")
            if not os.path.exists(save_visual_path):
                os.makedirs(save_visual_path)
            ext = ".png"
            for file_name in files:
                if ext in file_name and "._" not in file_name:
                    label = cv2.imread(os.path.join(root, file_name))
                    image = cv2.imread(
                        os.path.join(root, file_name).replace("label", "image")
                    )
                    visual = cv2.addWeighted(image, 0.8, label, 0.2, 0)
                    cv2.imwrite(os.path.join(save_visual_path, file_name), visual)

        if keys == ["label_roi"]:
            save_visual_roi_path = root.replace("label_roi", "visual_roi")
            if not os.path.exists(save_visual_roi_path):
                os.makedirs(save_visual_roi_path)
            ext = ".png"
            for file_name in files:
                if ext in file_name and "._" not in file_name:
                    label_roi = cv2.imread(os.path.join(root, file_name))
                    image_roi = cv2.imread(
                        os.path.join(root, file_name).replace("label_roi", "image_roi")
                    )
                    visual_roi = cv2.addWeighted(image_roi, 0.8, label_roi, 0.2, 0)
                    cv2.imwrite(
                        os.path.join(save_visual_roi_path, file_name), visual_roi
                    )

print("运行结束")

## 文件拷贝移动

In [ ]:
import os
import shutil

path = r"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/动态返修3"
save_path = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/动态返修3"

# 通过DFS实现统一逻辑处理同一层次的文件对象
for root, dirs, files in os.walk(path):
    if not dirs:
        keywords = ["visual", "label", "roi"]
        dir_name = root.split("/")[-1]
        keys = [key for key in keywords if key in dir_name]
        if not keys:
            save_image_path = os.path.join(save_path, "image", root.split("/")[-2])
            if not os.path.exists(save_image_path):
                os.makedirs(save_image_path)
            ext = ".png"
            for file_name in files:
                if ext in file_name:
                    shutil.copy(os.path.join(root, file_name), save_image_path)

        if keys == ["label"]:
            save_label_path = os.path.join(save_path, "label", root.split("/")[-2])
            if not os.path.exists(save_label_path):
                os.makedirs(save_label_path)
            ext = ".png"
            for file_name in files:
                if ext in file_name:
                    shutil.copy(os.path.join(root, file_name), save_label_path)

print("运行结束")

# 文件夹拷贝

In [13]:
import os
import shutil

path_roi_label = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/dataset_roi/label"
path_roi_image = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/dataset_roi/image"
path_label = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/label"
path_image = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/image"

save_dir = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset_usable_0815"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

keys_list = os.listdir(path_roi_label)

for dirs in os.listdir(path_label):
    if dirs not in keys_list:
        save_path = os.path.join(save_dir, dirs)
        # shutil.copytree(os.path.join(path_roi_label, dirs), os.path.join(save_path, "label_roi"))
        # shutil.copytree(os.path.join(path_roi_image, dirs).replace("label", "image"), os.path.join(save_path, "image_roi"))
        shutil.copytree(
            os.path.join(path_label, dirs), os.path.join(save_path, "label")
        )
        shutil.copytree(
            os.path.join(path_image, dirs).replace("label", "image"),
            os.path.join(save_path, "image"),
        )

print("运行结束")

运行结束


## *Get ROI data pair*

In [ ]:
import numpy as np
import os
import cv2
from skimage import morphology


def file_remove(file_list):
    try:
        file_list.remove(".DS_Store")
    except ValueError:
        pass
    try:
        file_list.remove("._.DS_Store")
    except ValueError:
        pass


def remove_isolate(inputs, threshold_area=0.5):
    """
    inputs: x*y*3
    """
    mask = np.zeros((inputs.shape[0], inputs.shape[1]), dtype=np.uint8)
    mask[np.sum(inputs, axis=-1) > 0] = 1
    mask = morphology.remove_small_objects(
        mask.astype(np.bool8), np.sum(mask) * threshold_area, connectivity=8
    ).astype(np.uint8)
    outputs = np.expand_dims(mask, -1) * inputs

    return outputs


def crop_array(image, label, up, down, left, right):
    """
    image:x*y*3 uint8
    label:x*y*3 uint8
    """

    crop_image = image[up:down, left:right, :]
    crop_label = label[up:down, left:right, :]

    return crop_image, crop_label


def count_list_place(base_path, files):
    place = []
    file_dir = f"{base_path}/{files}"
    path_4 = os.listdir(file_dir)
    file_remove(path_4)
    for file_name in path_4:
        if "._" not in file_name:
            data = cv2.imdecode(
                np.fromfile(f"{file_dir}/{file_name}", dtype=np.uint8), -1
            )

            data = remove_isolate(data)
            data_gray = cv2.cvtColor(data, cv2.COLOR_RGB2GRAY)

            y_sum = np.sum(data_gray, axis=1)
            x_sum = np.sum(data_gray, axis=0)

            up = (
                min(np.where(y_sum != 0)[0]) - 20
                if min(np.where(y_sum != 0)[0]) > 20
                else 0
            )
            down = (
                max(np.where(y_sum != 0)[0]) + 20
                if max(np.where(y_sum != 0)[0]) < data.shape[0] - 20
                else data.shape[0]
            )
            left = (
                min(np.where(x_sum != 0)[0]) - 20
                if min(np.where(x_sum != 0)[0]) > 20
                else 0
            )
            right = (
                max(np.where(x_sum != 0)[0]) + 20
                if min(np.where(x_sum != 0)[0]) < data.shape[1] - 20
                else data.shape[1]
            )

        place.append([up, down, left, right])

    place = np.array(place)
    place_min = np.min(place, axis=0)
    place_max = np.max(place, axis=0)
    up, down, left, right = place_min[0], place_max[1], place_min[2], place_max[3]
    assert (down < data.shape[0]) and (right < data.shape[1])

    return up, down, left, right


def count_place(data, path):
    data_gray = cv2.cvtColor(data, cv2.COLOR_RGB2GRAY)

    y_sum = np.sum(data_gray, axis=1)
    x_sum = np.sum(data_gray, axis=0)
    try:
        up = (
            min(np.where(y_sum != 0)[0]) - 20
            if min(np.where(y_sum != 0)[0]) > 20
            else 0
        )
        down = (
            max(np.where(y_sum != 0)[0]) + 20
            if max(np.where(y_sum != 0)[0]) < data.shape[0] - 20
            else data.shape[0]
        )
        left = (
            min(np.where(x_sum != 0)[0]) - 20
            if min(np.where(x_sum != 0)[0]) > 20
            else 0
        )
        right = (
            max(np.where(x_sum != 0)[0]) + 20
            if min(np.where(x_sum != 0)[0]) < data.shape[1] - 20
            else data.shape[1]
        )
        flag = 0
    except ValueError:
        flag = 1
        print(path)

    return up, down, left, right, flag


file_path = r"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/0816/3"
save_base = r"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/0816"
if not os.path.exists(save_base):
    os.mkdir(save_base)
save_base_label = f"{save_base}/label"
if not os.path.exists(save_base_label):
    os.mkdir(save_base_label)
save_base_image = f"{save_base}/image"
if not os.path.exists(save_base_image):
    os.mkdir(save_base_image)

for root, dirs, files in os.walk(file_path):
    if not dirs:
        keywords = ["label"]
        dir_name = root.split("/")[-2]
        keys = [key for key in keywords if key in dir_name]
        if keys:
            print(f"process {root}")
            ext = ".png"
            for file_name in files:
                if ext in file_name and "._" not in file_name:
                    file_path_label = os.path.join(root, file_name)
                    label = cv2.imdecode(
                        np.fromfile(file_path_label, dtype=np.uint8), -1
                    )
                    label = remove_isolate(label)
                    file_path_image = file_path_label.replace("label", "image")
                    image = cv2.imdecode(
                        np.fromfile(file_path_image, dtype=np.uint8), -1
                    )

                    up, down, left, right, flag = count_place(label, file_path_label)
                    if flag:
                        break

                    crop_image, crop_label = crop_array(
                        image, label, up, down, left, right
                    )
                    save_path_label = os.path.join(save_base_label, root.split("/")[-1])
                    if not os.path.exists(save_path_label):
                        os.mkdir(save_path_label)
                    save_path_image = os.path.join(save_base_image, root.split("/")[-1])
                    if not os.path.exists(save_path_image):
                        os.mkdir(save_path_image)

                    cv2.imencode(".png", crop_label)[1].tofile(
                        os.path.join(save_path_label, file_name)
                    )
                    cv2.imencode(".png", crop_image)[1].tofile(
                        os.path.join(save_path_image, file_name)
                    )

print("运行结束")

# 寻找重复名称文件

In [10]:
data_dir = list(
    open(
        "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/王昊数据_0801/已返修/数据名单.txt",
        "r",
    )
)

data_dict = {}
for path_1 in data_dir:
    path_list = []
    data_1 = path_1.split("/")[-1]
    for path_2 in data_dir:
        data_2 = path_2.split("/")[-1]
        if data_1 == data_2:
            path_list.append(path_2)

    data_dict[data_1] = path_list

print(data_dict)